## Importing Dataset

In [1]:
import pandas as pd
df = pd.read_csv('archive/kannada.csv')
df.head(2)

,img,class
0,Img/img001-001.png,1
1,Img/img001-002.png,1


In [2]:
# Changing class range from 0 to 656.
df['class'] = df['class'] - 1 

In [3]:
# Dataset Size
df.shape

(16425, 2)

In [4]:
# Total Numbers of Output Classes
len(df['class'].unique())

657

In [5]:
# Total Number of Images Per Class
df['class'].value_counts()

class
656    25
640    25
639    25
638    25
637    25
       ..
5      25
4      25
3      25
2      25
1      25
Name: count, Length: 657, dtype: int64

## Image Preprocessing

In [6]:
import numpy as np
import cv2
import os
def load_and_preprocess_images(image_paths):
    """This Function is used to preprocess the image such as grayscale convertion, resizing, normalization etc."""
    images = []
    img_dir = 'archive/'
    img_size=(64, 64)
    for img_path in image_paths:
        full_path = os.path.join(img_dir, img_path)
        img = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, img_size)
        img = img / 255.0
        images.append(img)
    return np.array(images)
image_paths = df["img"].values
labels = df["class"].values
X = load_and_preprocess_images(image_paths)

In [7]:
X

array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]],

       ...,

       [[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        ...,
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1.

In [8]:
labels

array([  0,   0,   0, ..., 656, 656, 656])

In [9]:
# Dividing X, labels into training and testing data.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.25, random_state=42)

In [10]:
# Reshape the data to fit the CNN model
X_train = X_train.reshape(-1, 64, 64, 1)
X_test = X_test.reshape(-1, 64, 64, 1)

## CNN Implementation 

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
model = Sequential([
        Conv2D(32, (3,3), activation='relu', input_shape=(64,64,1)), 
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Conv2D(64, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Conv2D(128, (3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.5),
        Dense(657, activation='softmax')
    ])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

c:\Users\ranuj\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [13]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=20,restore_best_weights=True)

In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
    rotation_range=20, 
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
)
datagen.fit(X_train)

In [15]:
model.fit(datagen.flow(X_train, y_train, batch_size=32), epochs=100, validation_data=(X_test, y_test), callbacks=[early_stopping_callback])

Epoch 1/100


c:\Users\ranuj\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


385/385 ━━━━━━━━━━━━━━━━━━━━ 72s 175ms/step - accuracy: 0.0026 - loss: 6.6465 - val_accuracy: 4.8697e-04 - val_loss: 6.7573
Epoch 2/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 67s 174ms/step - accuracy: 0.0047 - loss: 6.3970 - val_accuracy: 0.0068 - val_loss: 5.9893
Epoch 3/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 67s 174ms/step - accuracy: 0.0071 - loss: 6.1338 - val_accuracy: 0.0017 - val_loss: 9.8538
Epoch 4/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 67s 173ms/step - accuracy: 0.0116 - loss: 5.8432 - val_accuracy: 0.0056 - val_loss: 7.4289
Epoch 5/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 69s 178ms/step - accuracy: 0.0273 - loss: 5.4627 - val_accuracy: 0.0114 - val_loss: 6.7866
Epoch 6/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 74s 191ms/step - accuracy: 0.0497 - loss: 4.9870 - val_accuracy: 0.0565 - val_loss: 4.7391
Epoch 7/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 76s 197ms/step - accuracy: 0.0776 - loss: 4.5065 - val_accuracy: 0.0083 - val_loss: 9.2775
Epoch 8/100
385/385 ━━━━━━━━━━━━━━━━━━━━ 73s 190ms/step - accuracy: 0.1119 - loss: 4

In [18]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
from sklearn.metrics import accuracy_score
print(accuracy_score(y_pred_classes, y_test))

129/129 ━━━━━━━━━━━━━━━━━━━━ 4s 33ms/step
0.7302167031896761


In [19]:
model.save('cnn.h5')

In [16]:
from tensorflow.keras.models import load_model
model = load_model('cnn.h5')

In [25]:
# Feature Extraction from CNN Model.
from tensorflow.keras.models import Model
feature_extractor = Model(inputs=model.inputs, outputs=model.get_layer(index=9).output)
X_train_features = feature_extractor.predict(X_train)
X_test_features = feature_extractor.predict(X_test)

c:\Users\ranuj\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(32, 64, 64, 1))
  warnings.warn(msg)


384/385 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step

c:\Users\ranuj\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\models\functional.py:237: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: ['keras_tensor']
Received: inputs=Tensor(shape=(None, 64, 64, 1))
  warnings.warn(msg)


385/385 ━━━━━━━━━━━━━━━━━━━━ 20s 51ms/step
129/129 ━━━━━━━━━━━━━━━━━━━━ 7s 51ms/step


## KNN Implementation

In [26]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
knn = KNeighborsClassifier()
knn.fit(X_train_features, y_train)
y_pred_knn = knn.predict(X_test_features)
accuracy_knn = accuracy_score(y_test, y_pred_knn)
print(f"KNN Accuracy: {accuracy_knn}")

KNN Accuracy: 0.17384952520087654


## SVM Implementation

In [29]:
from sklearn.svm import SVC
svm = SVC(kernel='linear')
svm.fit(X_train_features, y_train)
y_pred_svm = svm.predict(X_test_features)
accuracy_svm = accuracy_score(y_test, y_pred_svm)
print(f"SVM Accuracy: {accuracy_svm}")

SVM Accuracy: 0.6094472851229608
